In [5]:
import numpy as np
import shutil
from scipy.interpolate import interp1d
import h5py

# function that will compute the effective density values to give to Rockstar 
def compute_effective_cosmology(
    a_snap,
    a_bg,
    rho_em,
    rho_lm,
    rho_rad,
    h0_true):
    
    '''arguments: provide the scale factor of the simulation,
    provide the "true" arrays of scale factor a, early matter, late matter, 
    radiation; and provide the true H0 (hubble constant) from simulation'''
    
    a_bg = np.asarray(a_bg)
    rho_em = np.asarray(rho_em)
    rho_lm = np.asarray(rho_lm)
    rho_rad = np.asarray(rho_rad)

    if np.any(np.diff(a_bg) <= 0):
        raise ValueError("a_bg must be strictly increasing.")

    #turn arrays into functions:
    kind = "cubic"
    f_em  = interp1d(a_bg, rho_em,  kind=kind, fill_value="extrapolate")
    f_lm  = interp1d(a_bg, rho_lm,  kind=kind, fill_value="extrapolate")
    f_rad = interp1d(a_bg, rho_rad, kind=kind, fill_value="extrapolate")


    # calculate densities & density parameters at scale factor of snapshot
    rho_em_snap  = float(f_em(a_snap))
    rho_lm_snap  = float(f_lm(a_snap))
    rho_rad_snap = float(f_rad(a_snap))

    rho_tot_snap = rho_em_snap + rho_lm_snap + rho_rad_snap

    Omegam_true_snap = (rho_em_snap + rho_lm_snap) / rho_tot_snap

    
    a = float(a_snap)
    y = Omegam_true_snap
    a3inv = a**-3
    # here is the calculation of the effective density values with which to feed rockstar
    Om0_eff = y / (a3inv - y * (a3inv - 1.0))
    Ol0_eff = 1.0 - Om0_eff


    f_tot = interp1d(a_bg, rho_em + rho_lm + rho_rad,
                     kind=kind, fill_value="extrapolate")

    rho_tot_today = float(f_tot(1.0)) 
    E_true_snap = np.sqrt(rho_tot_snap / rho_tot_today)
    h_eff = h0_true * E_true_snap

    return Om0_eff, Ol0_eff, h_eff, Omegam_true_snap



In [11]:
# these functions are for being able to read & deal with the gadget-format concept snapshot files!



# This dtype matches the classic GADGET-2/GADGET-3 header structure as written in
# the snapshot "HEAD" block. '<' means little-endian. 
# many of the fields are fixed-length arrays of 6 elements (one per particle type).
# this is used to interpret raw bytes from the snapshot and write the modified header back in-place.


gadget_header_dtype = np.dtype([
    ('npart',                ('<i4', 6)),   
    ('mass',                 ('<f8', 6)),   
    ('time',                 '<f8'),        
    ('redshift',             '<f8'),        
    ('flag_sfr',             '<i4'),        
    ('flag_feedback',        '<i4'),      
    ('npartTotal',           ('<i4', 6)),   
    ('flag_cooling',         '<i4'),        
    ('num_files',            '<i4'),        
    ('BoxSize',              '<f8'),       
    ('Omega0',               '<f8'),       
    ('OmegaLambda',          '<f8'),        
    ('HubbleParam',          '<f8'),       
    ('flag_stellarage',      '<i4'),       
    ('flag_metals',          '<i4'),        
    ('npartTotalHighWord',   ('<i4', 6)),   
    ('flag_entropy_instead_u','<i4'),       
    ('flag_doubleprecision', '<i4'),        
    ('flag_lpt_ics',         '<i4'),       
    ('lpt_scalingfactor',    '<f4'),       
    ('fill',                 '48b'),      
])




def _patch_header_struct(header, a_snap, Om0_eff, Ol0_eff, h_eff, update_time_and_redshift):
    '''this function will patch a parsed GADGET header struct
        (a 0-d numpy structured array element) with the updated "new"
        "effective cosmology", it is just a helper fn that helps keeps all
        file-format handling elsewhere and only modifies the relevant fields.
  
        args:
        header : numpy.void (structured)
            A single header record parsed using gadget_header_dtype.
        a_snap : float
            Snapshot scale factor to write to the header if update_time_and_redshift is True.
        Om0_eff, Ol0_eff : float
            effective Omega_m,0 and Omega_Λ,0 to store in header.
        h_eff : float
            effective h parameter to store in header (H0 / 100 km/s/Mpc).
        update_time_and_redshift : bool
            If True, also overwrite the header's time and redshift fields so they are
            consistent with a_snap.

        returns:
        header : numpy.void
            The same header record, now modified, for convenience.
        '''
    header['Omega0']      = Om0_eff
    header['OmegaLambda'] = Ol0_eff
    header['HubbleParam'] = h_eff
    if update_time_and_redshift:
        header['time']     = a_snap
        header['redshift'] = 1.0 / a_snap - 1.0
    return header


def patch_gadget_header_cosmology(
    infile,
    outfile,
    a_snap,
    Om0_eff,
    Ol0_eff,
    h_eff,
    update_time_and_redshift=True
):
    """
    patch gadget snapshot header in-place (or via copy) to store an "effective"
    cosmology (Ω_m,0, Ω_Λ,0, h). optionally also update header time/redshift.

    This function supports:
      - GADGET format 1: header appears immediately after an initial 4-byte block size
        marker equal to 256.
      - GADGET format 2: first comes a named block header with a 4-char block name
        (expects HEAD/HEADER first), then a data block containing the header bytes.

    Parameters
    ----------
    infile : str or path-like
        Path to the input snapshot file.
    outfile : str or path-like
        Path to the output snapshot file. If different from infile, infile is copied first.
        If the same, the snapshot is modified in-place.
    a_snap : float
        Scale factor to optionally store in the header.
    Om0_eff, Ol0_eff : float
        Effective present-day Ω_m,0 and Ω_Λ,0 to store in the header.
    h_eff : float
        Effective h parameter to store in the header.
    update_time_and_redshift : bool, default True
        If True, also overwrite 'time' and 'redshift' based on `a_snap`.

    """

    # If writing to a different output path, copy the original first so we can
    # safely patch the copy without touching the original.
    if infile != outfile:
        shutil.copy2(infile, outfile)

    # Open in read+binary mode so we can both read and overwrite bytes.
    with open(outfile, "r+b") as f:
        # Peek at the first 4 bytes to decide format.
        # - In format-1, this is typically 256 (the header block size).
        # - In format-2, this is typically 8 (the size of the block-name wrapper).
        first = np.fromfile(f, dtype='<i4', count=1)
        if first.size != 1:
            raise IOError("Could not read first 4 bytes from snapshot.")

        first_val = int(first[0])

        # -----------------------------
        # FORMAT 1: first int is the header block-size marker (256).
        # Layout: [int32 blocklen=256][256 bytes header][int32 blocklen=256]...
        # We read the header struct immediately and then overwrite it starting at byte 4.
        if first_val == 256:
            header_arr = np.fromfile(f, dtype=gadget_header_dtype, count=1)
            if header_arr.size != 1:
                raise IOError("Could not read GADGET header (format 1).")
            header = header_arr[0]

            # Patch cosmology (and optionally time/redshift).
            header = _patch_header_struct(
                header, a_snap, Om0_eff, Ol0_eff, h_eff, update_time_and_redshift
            )

            # Seek to the start of the header bytes (right after the initial 4-byte marker)
            # and overwrite with the patched header record.
            f.seek(4)
            np.array([header], dtype=gadget_header_dtype).tofile(f)
            return


        
        # -----------------------------
        # FORMAT 2: first int is 8, followed by:
        # [int32=8][4 chars blockname][int32=...][int32=8][int32 data_blocksize][data...]
        # This implementation assumes the first named block is HEAD/HEADER.
        elif first_val == 8:
            # Read the 4-character block name (e.g., "HEAD").
            blockname_bytes = f.read(4)
            if len(blockname_bytes) != 4:
                raise IOError("Could not read blockname for format 2 snapshot.")
            blockname = blockname_bytes.decode('ascii', errors='ignore')

            # Skip length / wrapper bookkeeping for the named block header.
            skip_len_arr = np.fromfile(f, dtype='<i4', count=1)
            if skip_len_arr.size != 1:
                raise IOError("Could not read skip length for format 2 snapshot.")
            skip_len = int(skip_len_arr[0])

            # End-of-name-block marker should be 8 to close the name wrapper.
            end_block_arr = np.fromfile(f, dtype='<i4', count=1)
            if end_block_arr.size != 1:
                raise IOError("Could not read end-of-name-block marker.")
            if int(end_block_arr[0]) != 8:
                raise ValueError(
                    f"Unexpected end-of-name-block marker: {end_block_arr[0]} (expected 8)."
                )

            # Sanity check: this patcher assumes the first named block is the header block.
            if blockname.strip().upper() not in ("HEAD", "HEADER"):
                raise ValueError(
                    f"First block name is '{blockname}', not 'HEAD'; "
                    "this patcher assumes HEAD is the first block."
                )

            # Next is the size (in bytes) of the header data block.
            data_blocksize_arr = np.fromfile(f, dtype='<i4', count=1)
            if data_blocksize_arr.size != 1:
                raise IOError("Could not read header data block size.")
            data_blocksize = int(data_blocksize_arr[0])

            # Remember where the header data starts so we can seek back and overwrite.
            data_start_pos = f.tell()

            # Read the entire header data block into a mutable buffer.
            raw = bytearray(f.read(data_blocksize))
            if len(raw) != data_blocksize:
                raise IOError("Could not read full header data block.")

            # Ensure the block contains at least one full header struct.
            needed = gadget_header_dtype.itemsize
            if data_blocksize < needed:
                raise ValueError(
                    f"Header data block ({data_blocksize} bytes) smaller than expected "
                    f"GADGET header size ({needed} bytes)."
                )

            # Interpret the first `needed` bytes as a header struct.
            header = np.frombuffer(raw[:needed], dtype=gadget_header_dtype, count=1)[0]

            # Patch cosmology (and optionally time/redshift).
            header = _patch_header_struct(
                header, a_snap, Om0_eff, Ol0_eff, h_eff, update_time_and_redshift
            )

            # Convert the patched header back into bytes and replace in the raw buffer.
            header_bytes = np.array([header], dtype=gadget_header_dtype).tobytes()
            raw[:needed] = header_bytes

            # Seek back and overwrite the header data block with the modified buffer.
            f.seek(data_start_pos)
            f.write(raw)

            return

        # -----------------------------
        # If neither marker matches, we don't know how to safely parse the file.
        else:
            raise ValueError(
                f"Unrecognized first int in snapshot: {first_val}. "
                "Expected 256 (format 1) or 8 (format 2)."
            )


def patch_snapshot(
    snapshot_in,
    snapshot_out,
    a_snap,
    a_bg,
    rho_em,
    rho_lm,
    rho_rad,
    h0_true
):
    """
   combines all above functions to compute an "effective" cosmology at given
    snapshot scale factor, print a quick summary, and then modify the gadget header accordingly.
    args:
    snapshot_in : str or path-like
        Input snapshot path.
    snapshot_out : str or path-like
        Output snapshot path (can be the same as input for in-place patching).
    a_snap : float
        Scale factor of the snapshot you want written into the header (and used for redshift).
    a_bg, rho_em, rho_lm, rho_rad, h0_true : various
        Passed through to `compute_effective_cosmology(...)`. This function is expected
        to return (Om0_eff, Ol0_eff, h_eff, Om_true).

    """
    # Compute a ΛCDM-equivalent parameter set to store in the header,
    # plus the "true" matter fraction for logging.
    Om0_eff, Ol0_eff, h_eff, Om_true = compute_effective_cosmology(
        a_snap, a_bg, rho_em, rho_lm, rho_rad, h0_true
    )

    # Quick log so you can confirm what was computed before modifying the file.
    print(f"a_snap={a_snap:.4f}, Ωm_true={Om_true:.5f}, "
          f"Ωm0_eff={Om0_eff:.5f}, ΩΛ0_eff={Ol0_eff:.5f}, h_eff={h_eff:.5f}")

    # Patch the snapshot header (and also ensure time/redshift fields match a_snap).
    patch_gadget_header_cosmology(
        snapshot_in,
        snapshot_out,
        a_snap,
        Om0_eff,
        Ol0_eff,
        h_eff,
        update_time_and_redshift=True,
    )


# sanity check function
def read_gadget_header_basic(path):
    with open(path, "rb") as f:
        first = np.fromfile(f, dtype="<i4", count=1)
        first_val = int(first[0])

        if first_val == 256:
            # format 1
            header_arr = np.fromfile(f, dtype=gadget_header_dtype, count=1)
            header = header_arr[0]
        elif first_val == 8:
            # format 2: skip blockname block
            blockname_bytes = f.read(4)
            skip_len = np.fromfile(f, dtype="<i4", count=1)[0]
            end_block = np.fromfile(f, dtype="<i4", count=1)[0]
            data_blocksize = np.fromfile(f, dtype="<i4", count=1)[0]
            raw = f.read(data_blocksize)
            header = np.frombuffer(raw[:gadget_header_dtype.itemsize],
                                   dtype=gadget_header_dtype, count=1)[0]
        else:
            raise ValueError(f"Unrecognized first int: {first_val}")

    print("Omega0      =", header['Omega0'])
    print("OmegaLambda =", header['OmegaLambda'])
    print("HubbleParam =", header['HubbleParam'])
    print("time        =", header['time'])
    print("redshift    =", header['redshift'],"\n")

In [9]:
# now get background quantities to use as the "true" values to compute the effective values
# update file here with your class_processed.hdf5 file from CONCEPT
file = '/mnt/c/Users/doman/OneDrive/Desktop/class_processed.hdf5'


with h5py.File(file, "r") as f:
    bg = f["background"]

    t = bg["t"][:]   # cosmic time [Gyr by default]
    a_bg = bg["a"][:]   # scale factor
    z = bg["z"][:]   # redshift
    
    print('Available background quantities:', list(bg.keys()))

    # store background quantities & densities, get density params
    rho_cdm = bg["rho_cdm"][:]
    rho_crit = bg["rho_crit"][:]
    rho_tot = bg["rho_tot"][:]
    rho_b = bg["rho_b"][:]
    rho_dcdm = bg["rho_dcdm"][:]
    rho_dr = bg["rho_dr"][:]

    Odcdm = rho_dcdm / rho_crit
    Ocdm = rho_cdm / rho_crit
    Ob = rho_b / rho_crit
    Orad = rho_dr / rho_crit


Available background quantities: ['D', 'D2', 'D3a', 'D3b', 'D3c', 'H', 'a', 'f', 'f2', 'f3a', 'f3b', 'f3c', 'p_b', 'p_cdm', 'p_dcdm', 'p_dr', 'p_g', 'p_lambda', 'p_lapse', 'p_metric', 'p_tot', 'p_ur', 'rho_b', 'rho_cdm', 'rho_crit', 'rho_dcdm', 'rho_dr', 'rho_g', 'rho_lambda', 'rho_lapse', 'rho_metric', 'rho_tot', 'rho_ur', 't', 'tau', 'z']


In [13]:
# example usage
# run this on all snapshot files for which you want to modify the background cosmology values with the "effective" values. 
# it will create a new modified snapshot file and leave the original one unchanged

# update these file names to the original snapshot and the name of the new modified effective snapshot file:
snapshot_og = "/mnt/c/Users/doman/OneDrive/Desktop/dcdmtut3/snapshot_a=0.60"
snapshot_eff = "/mnt/c/Users/doman/OneDrive/Desktop/dcdmtut3/snapshot_a=0.60_eff"

a_snap = 0.6 # edit this to be the scale factor of the snapshot being modified
# fill in the scale factor, early matter, late matter, and radiation (decay products) densities, and they need to be array-like
a = a_bg
em = rho_dcdm
lm = rho_cdm
rad = rho_dr
# include the true h0 of the simulation
h0 = 0.67


# print values of original snapshot file
print("Before:")
read_gadget_header_basic(snapshot_og)

patch_snapshot(
    snapshot_in=snapshot_og,     # original from CONCEPT
    snapshot_out=snapshot_eff, # patched for Rockstar usage
    a_snap=a_snap,  
    a_bg=a,
    rho_em= em,
    rho_lm= lm,
    rho_rad= rad,
    h0_true=h0
)

# print new values to double check the change
print("\nAfter:")
read_gadget_header_basic(snapshot_eff)


Before:
Omega0      = 0.3190002782200307
OmegaLambda = 0.6809997217799693
HubbleParam = 0.67
time        = 0.5999999999992743
redshift    = 0.6666666666686825 

a_snap=0.6000, Ωm_true=0.60790, Ωm0_eff=0.25087, ΩΛ0_eff=0.74913, h_eff=1.63660

After:
Omega0      = 0.2508688647368418
OmegaLambda = 0.7491311352631582
HubbleParam = 1.6365990908041106
time        = 0.6
redshift    = 0.6666666666666667 

